In [23]:
#!/usr/bin/env python
import snowflake.connector

import time
import calendar

import hashlib as hashlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics as sm

from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from mpl_toolkits.mplot3d import Axes3D

from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5

from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [13]:
#---------------------------------------
# For installing dependencies
#---------------------------------------
# import sys
# !{sys.executable} -m pip install "snowflake-connector-python[pandas]"
# !{sys.executable} -m pip install "matplotlib"
# !{sys.executable} -m pip install "scikit-learn"

In [5]:
#---------------------------------------
# CONNECT TO SNOWFLAKE
#---------------------------------------

con = snowflake.connector.connect(
    user='codestudio_samplefarm_evening',
    password='ikRsb9D00UNr',
    account='FT76388.ap-southeast-2',
    #warehouse='Main',
    database='AW_INTERNAL_STAGING',
    schema='AW_AURORA_AGRIWEBBDATA'
    )

In [20]:
#---------------------------------------
# Test reaching the dataset
#---------------------------------------
con_cursor = con.cursor()
try:
    con_cursor.execute("SELECT UUID, FARMID, TENANTID, TITLE, PARENTUUID, COLOR, ORIGIN, DATEOFBIRTH, \
                       PURCHASEPRICE, LONGITUDE, LATITUDE, SUBTYPE, BREEDS, ASSETTYPE, WEIGHT,\
                        PASTUREID, SIZE \
                        \
                        FROM AW_INTERNAL_STAGING.AW_AURORA_AGRIWEBBDATA.MOBLIVESTOCKGROUP")
    
    pd.set_option("max_rows", 1000)
    
    df = con_cursor.fetch_pandas_all()
    
    print(df)
    
finally:
    con_cursor.close()

                                        UUID  \
0       78cb1f49-688f-4a09-8d1c-f8d744e0b4cd   
1       2bec4b6c-2d0a-4103-b309-bf055a1cd094   
2       85432637-09bf-4538-b441-07a8975ef641   
3       949d0755-0ff2-45d7-a40d-b38d0df56470   
4       f7a68a8d-5085-4a35-8b93-4d299444b872   
...                                      ...   
199995  12f8792c-3cfb-45dc-9fc8-1be4788a3443   
199996  12f8bf8b-e3b7-4c41-9ee2-aabb24983b41   
199997  12f8f08f-d1ba-46cd-8ad5-4f8ebf6d5c92   
199998  1283fa52-032a-4427-b4ca-315792f6d9eb   
199999  12f93ef2-e333-4a3f-a103-ede8bac622af   

                                      FARMID  \
0       fa380d77-1136-4427-a657-7a23cb7ed3d0   
1       fa380d77-1136-4427-a657-7a23cb7ed3d0   
2       fa380d77-1136-4427-a657-7a23cb7ed3d0   
3       fa380d77-1136-4427-a657-7a23cb7ed3d0   
4       fa380d77-1136-4427-a657-7a23cb7ed3d0   
...                                      ...   
199995  4f3501ea-eb6b-4b15-9cde-424e9535c0f1   
199996  d32b74a4-f9ac-4497-b907-044a2af

In [43]:
#---------------------------------------
# Test clustering algorithm
#---------------------------------------
clustering = KMeans(n_clusters=3, random_state=10)

con_cursor = con.cursor()
try:
    con_cursor.execute("SELECT COLOR, ORIGIN, DATEOFBIRTH, PURCHASEPRICE, LONGITUDE, LATITUDE, SUBTYPE, BREEDS, WEIGHT, SIZE \
                        FROM AW_INTERNAL_STAGING.AW_AURORA_AGRIWEBBDATA.MOBLIVESTOCKGROUP")
    
    pd.set_option("max_rows", 1000)
    
    df = con_cursor.fetch_pandas_all()
    
#     # Preprocessing
#     enc = OneHotEncoder()
#     enc.fit(df)
    
#     enc_df = pd.DataFrame(enc.fit_transform(df[['ORIGIN']]).toarray())
#     # merge with main df bridge_df on key values
#     df = df.join(enc_df)
#     print(df)
    
    origin_df = pd.get_dummies(df, columns=["ORIGIN", "SUBTYPE", "BREEDS"], 
                               prefix=["Origin_", "Subtype_", "Breeds_"], drop_first=True)
    
    df.merge(origin_df, how='left')
    
    print(df)
    
    #clustering.fit(df)
    
finally:
    con_cursor.close()

MemoryError: Unable to allocate 25.0 GiB for an array with shape (2814, 9539968) and data type uint8